In [37]:
import h5py
import numpy as np
import os

In [38]:
fast5_dir_name = 'Dane/squiggled'

In [39]:
fast5_file_list = os.listdir(fast5_dir_name)

In [40]:
fast5_file_list[0]

'DEAMERNANOPORE_20161117_FNFAB43577_MN16450_sequencing_run_MA_821_R9_4_NA12878_11_17_16_88738_ch454_read100_strand.fast5'

In [41]:
test_file =  h5py.File(os.path.join(fast5_dir_name, fast5_file_list[0]),'r')

In [42]:
corrected_events = test_file['Analyses/RawGenomeCorrected_000/BaseCalled_template/Events']
raw_signal = test_file['Raw/Reads']
normalization_info = test_file['Analyses/RawGenomeCorrected_000/BaseCalled_template'].attrs

In [43]:
def find_signal(name):
    if 'Signal' in name:
        return name

In [44]:
path = raw_signal.visit(find_signal)

In [45]:
dataset = raw_signal[path]

In [51]:
offset = corrected_events.attrs.get('read_start_rel_to_raw')
raw_signal = dataset[offset:]

In [47]:
lower_lim = normalization_info.get('lower_lim')
norm_type = normalization_info.get('norm_type')
outlier_threshold = normalization_info.get('outlier_threshold')
scale = normalization_info.get('scale')
shift = normalization_info.get('shift')
upper_lim = normalization_info.get('upper_lim')

In [52]:
np.median(raw_signal) == shift

True

In [53]:
scale == np.median(np.abs(raw_signal - shift))

True

In [55]:
normalized_signal = (raw_signal - shift) / scale

In [56]:
normalized_signal = np.where(normalized_signal > upper_lim, upper_lim, normalized_signal)

In [57]:
normalized_signal = np.where(normalized_signal < lower_lim, lower_lim, normalized_signal)

In [66]:
for row in corrected_events:
    if not row[0] == np.mean(normalized_signal[row[2]:row[2]+row[3]]) or not row[1] == np.std(normalized_signal[row[2]:row[2]+row[3]]):
        print('Error')